In [5]:
from z3 import *
def compute_tobacco_alcohol_tax(category, subcategory, quantity, alcohol_content=None, free_vars=None):
    """
    計算菸酒稅的應納稅額，並支援指定參數為自由變數以求最小稅額。
    free_vars: list of strings, e.g. ['quantity','alcohol_content']
    回傳: best_tax, fixed_tax, final_params, differences
    """
    # Z3 優化器
    opt = Optimize()
    # 定義 Z3 變數
    quantity_z = Real('quantity')
    alcohol_z = Real('alcohol_content')
    tax_z = Real('tax')

    # 參數字典：參數名稱 -> (Z3變數, 原始值, 約束)
    params = {'quantity': (quantity_z, quantity, [lambda v: v >= 0])}
    # 若計算酒類且需考慮酒精成分，加入該參數
    if category == '酒' and subcategory in [2,3,5,8]:
        if alcohol_content is None:
            raise ValueError("酒類計算需提供酒精成分")
        params['alcohol_content'] = (alcohol_z, alcohol_content, [lambda v: v >= 0])

    free_vars = free_vars or []
    # 套用參數約束
    for name, (var, val, cons) in params.items():
        if name in free_vars:
            for c in cons:
                opt.add(c(var))
        else:
            opt.add(var == val)
            for c in cons:
                opt.add(c(var))

    # 建立稅額表達式
    if category == '菸':
        # 健康福利捐固定1000
        health = 1000
        if subcategory in [1,5]:
            base = 1590 if subcategory == 1 else 590
            unit_div = 1000
        elif subcategory in [2,3,4,6,7,8]:
            base = 1590 if subcategory in [2,3,4] else 590
            unit_div = 1
        else:
            raise ValueError("菸類子類別需為1~8")
        tax_expr = (base + health) * (quantity_z / unit_div)
    elif category == '酒':
        # 使用 Z3 If 條件
        rate = If(subcategory == 1, 26,
               If(subcategory == 2, 7 * alcohol_z,
               If(subcategory == 3, 2.5 * alcohol_z,
               If(subcategory == 4, 185,
               If(subcategory == 5, 7 * alcohol_z,
               If(subcategory == 6, 9,
               If(subcategory == 7, 22,
               If(subcategory == 8, 7 * alcohol_z,
               If(subcategory == 9, 15,
               If(subcategory == 10, 11, 0))))))))))
        tax_expr = rate * quantity_z
    else:
        raise ValueError("類別須為 '菸' 或 '酒'")

    # 加入稅額約束
    opt.add(tax_z == tax_expr)

    # 計算初始固定稅額
    if free_vars:
        opt.push()
        for name in free_vars:
            var, orig, _ = params[name]
            opt.add(var == orig)
        if opt.check() == sat:
            fixed_tax = opt.model()[tax_z].as_long()
        else:
            fixed_tax = None
        opt.pop()
    else:
        if opt.check() == sat:
            fixed_tax = opt.model()[tax_z].as_long()
        else:
            fixed_tax = None

    print(f"初始固定稅額 (未放行自由變數) = {fixed_tax}")
    print(f"可調整變數: {free_vars}")

    # 開始最佳化
    best_tax = None
    iteration = 0
    while True:
        if opt.check() != sat:
            print(f"第 {iteration+1} 次最佳化: 無解，停止")
            break
        m = opt.model()
        cur = m[tax_z].as_long()
        if iteration == 0 or cur < best_tax:
            best_tax = cur
            print(f"----- 第 {iteration+1} 次最佳化 -----")
            print(f"當前稅額: {cur}")
            diffs = {}
            for name, (var, orig, _) in params.items():
                try:
                    vopt = m.eval(var).as_long()
                except:
                    vopt = None
                if vopt is not None and vopt != orig:
                    diffs[name] = {'original': orig, 'optimized': vopt, 'difference': vopt - orig}
            if diffs:
                print("變更的變數:")
                for k,d in diffs.items():
                    print(f"  {k}: 原始={d['original']}, 優化後={d['optimized']}, 差異={d['difference']}")
            print("-------------------------")
        else:
            print(f"第 {iteration+1} 次最佳化: 無變化，停止")
            break
        opt.add(tax_z < best_tax)
        iteration += 1

    # 整理最終結果
    final_params = {}
    differences = {}
    for name, (var, orig, _) in params.items():
        try:
            vopt = m.eval(var).as_long()
        except:
            vopt = None
        final_params[name] = {'value': vopt, 'type': 'free' if name in free_vars else 'fixed'}
        if vopt is not None and vopt != orig:
            differences[name] = {'original': orig, 'optimized': vopt, 'difference': vopt - orig}

    return best_tax, fixed_tax, final_params, differences

# 範例 1：菸類 – 紙菸（新制，subcategory=1），quantity = 10000 支，不放行自由變數
# 預期固定稅額 = (1590 + 1000) * (10000/1000) = 2590 * 10 = 25900
best_tax, fixed_tax, final_params, diffs = compute_tobacco_alcohol_tax('菸', 1, 10000)
print("Example 1 – fixed:", best_tax, fixed_tax, final_params, diffs)


# 範例 2：菸類 – 紙菸（新制），quantity = 10000 支，放行 quantity 作為自由變數
# 這時函式會找出最小可能的 quantity（>=0）使稅額最小，理論上會被優化到 0
best_tax, fixed_tax, final_params, diffs = compute_tobacco_alcohol_tax('菸', 1, 10000,
                                                      free_vars=['quantity'])
print("Example 2 – free quantity:", best_tax, fixed_tax, final_params, diffs)


# 範例 3：酒類 – 啤酒（subcategory=1），quantity = 500 公升，不需 alcohol_content，固定計算
# 稅率 = 26 元／公升，固定稅額 = 26 * 500 = 13000
best_tax, fixed_tax, final_params, diffs = compute_tobacco_alcohol_tax('酒', 1, 500, alcohol_content=0)
print("Example 3 – beer fixed:", best_tax, fixed_tax, final_params, diffs)


# 範例 4：酒類 – 蒸餾酒（subcategory=3），quantity = 200 公升，酒精度 40 度，
# 放行 alcohol_content 作為自由變數，優化結果會將度數降到 0 以獲得最小稅額
best_tax, fixed_tax, final_params, diffs = compute_tobacco_alcohol_tax('酒', 3, 200,
                                                      alcohol_content=40,
                                                      free_vars=['alcohol_content'])
print("Example 4 – free alcohol_content:", best_tax, fixed_tax, final_params, diffs)


初始固定稅額 (未放行自由變數) = 25900
可調整變數: []
----- 第 1 次最佳化 -----
當前稅額: 25900
-------------------------
第 2 次最佳化: 無解，停止
Example 1 – fixed: 25900 25900 {'quantity': {'value': 10000, 'type': 'fixed'}} {}
初始固定稅額 (未放行自由變數) = 25900
可調整變數: ['quantity']
----- 第 1 次最佳化 -----
當前稅額: 0
變更的變數:
  quantity: 原始=10000, 優化後=0, 差異=-10000
-------------------------
第 2 次最佳化: 無解，停止
Example 2 – free quantity: 0 25900 {'quantity': {'value': 0, 'type': 'free'}} {'quantity': {'original': 10000, 'optimized': 0, 'difference': -10000}}
初始固定稅額 (未放行自由變數) = 13000
可調整變數: []
----- 第 1 次最佳化 -----
當前稅額: 13000
-------------------------
第 2 次最佳化: 無解，停止
Example 3 – beer fixed: 13000 13000 {'quantity': {'value': 500, 'type': 'fixed'}} {}
初始固定稅額 (未放行自由變數) = 20000
可調整變數: ['alcohol_content']
----- 第 1 次最佳化 -----
當前稅額: 0
變更的變數:
  alcohol_content: 原始=40, 優化後=0, 差異=-40
-------------------------
第 2 次最佳化: 無解，停止
Example 4 – free alcohol_content: 0 20000 {'quantity': {'value': 200, 'type': 'fixed'}, 'alcohol_content': {'value': 0, 'type': 'fr